In [5]:
import os
import arcpy
import numpy as np
from datetime import datetime

inVector = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\White-Goose-Light-Pollution\Data\Vector\points.shp')
inRasterFolder = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\Images')
outRasterFolder = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\White-Goose-Light-Pollution\Data\Images_Extent')

arcpy.env.workspace = inRasterFolder

## getting extent and adding 1000m
desc = arcpy.Describe(inVector)
extent = desc.extent
bufferDistance = 0.0001388  ##1000m in degree units
extentString = str(extent.XMin - bufferDistance) + ' ' + str(extent.YMin - bufferDistance) + ' ' +  \
                str(extent.XMax + bufferDistance) + ' ' + str(extent.YMax + bufferDistance) 
print(extentString)


###clipping each raster
inRasters = arcpy.ListRasters()
for raster in inRasters:
    rasterDesc =  arcpy.Describe(raster)
    rasterName = rasterDesc.name
    outRaster = os.path.join(outRasterFolder, rasterName)
    
    ##delete raster if exits
    if arcpy.Exists(outRaster):
        arcpy.Delete_management(outRaster)
    
    ## clip raster
    arcpy.Clip_management (raster, extentString, outRaster, \
                     maintain_clipping_extent = 'NO_MAINTAIN_EXTENT')

3.6740312 51.1431912 87.6671388 73.5753088


In [6]:
def checkBreedingTime(date):
    for year in range(2006 , 2010):
        breedingStartTime = datetime(year, 5, 24)
        breedingEndTime = datetime(year, 6, 7)
        if date > breedingStartTime and date < breedingEndTime:
            isBreeding = 1
            break
        else:
            isBreeding = 0
    return isBreeding

In [34]:
###classifying breeding season and adding time and date fields
listFields = [field.name for field in arcpy.ListFields(inVector)]
checkFields = {'isBreed':'SHORT', 'time':'DATE', 'date':'DATE'}
for field, fieldType in checkFields.items():
    if field in listFields:
        arcpy.DeleteField_management(inVector, field)
    arcpy.AddField_management(inVector, field, fieldType)

fields = ['SHAPE@XY', 'timestamp', 'isBreed', 'date']

with arcpy.da.UpdateCursor(inVector,fields) as cursor:
    for row in cursor:
        time = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S')
        row[2] = checkBreedingTime(time)
        row[3] = time.date()
        cursor.updateRow(row)

arcpy.ConvertTimeField_management(inVector, 'timestamp', "yyyyMMddHHmmss", 'time')

inVectorArray = arcpy.da.FeatureClassToNumPyArray(inVector, ('isBreed'))
breedingCount = (inVectorArray['isBreed'] == 1).sum()
nonBreedingCount = (inVectorArray['isBreed'] == 0).sum()
print('Breeding points: ', breedingCount)
print('Non Breeding points: ', nonBreedingCount)

ExecuteError: ERROR 002126: Unable to convert all of the values from timestamp to time.
Failed to execute (ConvertTimeField).


In [25]:
hola = datetime.strptime('2012-01-10 00:00:00', '%Y-%m-%d %H:%M:%S')
hola.time()

datetime.time(0, 0)